In [2]:
import pandas as pd
import numpy as np
from tkinter import Tk, filedialog, simpledialog, Button, Frame, Label, Entry, messagebox, colorchooser, DISABLED, NORMAL, Toplevel
from tkinter.ttk import Progressbar
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from matplotlib.colors import LinearSegmentedColormap
from scipy.stats import circmean, circvar, circstd
import osmnx as ox
import time

# ASCII art for a rose
ROSE_ASCII = """
              @
        @@@@ @@ @@ 
     @@ @@@@@ @@@@@@ 
   @@@ @@@@@@@ @@@@@@@
  @@@@ @@@@@@@@ @@@@@@@@
 @@@@@ @@@@@@@@@ @@@@@@@@@
 @@@@@@ @@@@@@@@@@ @@@@@@@@@@
  @@@@@@ @@@@@@@@@ @@@@@@@@@@
  @@@@@@@ @@@@@@@@ @@@@@@@@@@
  @@@@@@@@@ @@@@@@ @@@@@@@@@@
  @@@@@@@@@@@ @@@ @@@@@@@@@@
  @@@@@@@@@@@@@@@ @@@@@@@@@@
   @@@@@@@@@@@@@ @@@@@@@@@@
     @@@@@@@@@@@ @@@@@@@@
        @@@@@@@@ @@@@@@
            @@@ @@@@
                @@
"""

WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600

default_cmap = LinearSegmentedColormap.from_list('hot_to_cold', [(1, 0, 0), (1, 1, 0), (0, 1, 0), (0, 1, 1), (0, 0, 1)])

def choose_color():
    color_code = colorchooser.askcolor(title="Choose color")[1]
    return [color_code] if color_code else default_cmap

def calculate_angle(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    angle_rad = np.arctan2(dy, dx)
    return np.degrees(angle_rad) % 360

def calculate_mean_direction(angles):
    mean_direction = circmean(np.radians(angles), high=2 * np.pi, low=0)
    return np.degrees(mean_direction)

def calculate_concentration(angles):
    return np.sqrt(1 - circvar(np.radians(angles)))

def calculate_circular_variance(angles):
    return circvar(np.radians(angles), high=2 * np.pi, low=0)

def calculate_mean_direction_display(angles):
    messagebox.showinfo("Mean Direction", f"Mean Direction: {calculate_mean_direction(angles):.2f}°")

def calculate_concentration_display(angles):
    messagebox.showinfo("Concentration", f"Concentration: {calculate_concentration(angles):.2f}")

def calculate_circular_variance_display(angles):
    messagebox.showinfo("Circular Variance", f"Circular Variance: {calculate_circular_variance(angles):.2f}")

def open_excel():
    file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx;*.xls")])
    return pd.read_excel(file_path) if file_path else None

def export_plot(fig):
    file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png")])
    if file_path:
        fig.savefig(file_path, dpi=400)
        messagebox.showinfo("Export Successful", "Plot exported successfully as PNG with 250 DPI.")

def export_statistics(angles, bars):
    file_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
    if file_path:
        with open(file_path, 'w') as f:
            f.write(f"Mean Direction: {calculate_mean_direction(angles):.2f}°\n")
            f.write(f"Concentration: {calculate_concentration(angles):.2f}\n")
            f.write(f"Circular Variance: {calculate_circular_variance(angles):.2f}\n\n")
            
            direction_ranges, bin_edges = np.histogram(angles, bins=360, range=(0, 360))
            f.write("Frequencies of data points in each direction range:\n")
            for i, freq in enumerate(direction_ranges):
                f.write(f"Direction range {i+1}°: {freq}\n")

        messagebox.showinfo("Export Successful", "Statistics exported successfully as TXT.")

def export_plot_transparent(fig):
    file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png")])
    if file_path:
        fig.savefig(file_path, dpi=400, transparent=True)
        messagebox.showinfo("Export Successful", "Transparent plot exported successfully as PNG with 250 DPI.")

def plot_rose(df, title, chosen_color, progress, label):
    angles = []
    total_rows = len(df)
    start_time = time.time()

    for index, row in df.iterrows():
        angles.append(calculate_angle(*row[['x1', 'y1', 'x2', 'y2']]))
        progress['value'] = (index + 1) / total_rows * 100
        label.config(text=f"Processing... {progress['value']:.2f}%")
        root.update_idletasks()
    
    end_time = time.time()
    elapsed_time = end_time - start_time

    fig = Figure(figsize=(8, 6), dpi=100)
    ax = fig.add_subplot(111, projection='polar')
    
    angles_radians = np.radians(angles)
    if isinstance(chosen_color, LinearSegmentedColormap):
        bars = ax.hist(angles_radians, bins=36, color='skyblue', edgecolor='black', linewidth=1.2)
        for bar, angle in zip(bars[2], np.linspace(0, 2 * np.pi, len(bars[0]), endpoint=False)):
            bar.set_facecolor(chosen_color(angle / (2 * np.pi)))
    else:
        bars = ax.hist(angles_radians, bins=36, color=chosen_color, edgecolor='black', linewidth=1.2)

    ax.set_theta_direction(-1)
    ax.set_theta_zero_location('N')
    ax.set_title(title if title else 'Rose Diagram of Line Orientations')

    label.config(text=f"Completed in {elapsed_time:.2f} seconds")
    progress['value'] = 100

    return fig, angles, bars

def plot_osm_rose(place_name, title, chosen_color, progress, label):
    G = ox.graph_from_place(place_name, network_type='all')
    edges = ox.graph_to_gdfs(G, nodes=False, edges=True)

    angles = []
    total_rows = len(edges)
    start_time = time.time()

    for index, row in enumerate(edges.itertuples(), 1):
        angles.append(calculate_angle(row.geometry.coords[0][0], row.geometry.coords[0][1],
                                      row.geometry.coords[-1][0], row.geometry.coords[-1][1]))
        progress['value'] = index / total_rows * 100
        label.config(text=f"Processing... {progress['value']:.2f}%")
        root.update_idletasks()
    
    end_time = time.time()
    elapsed_time = end_time - start_time

    angles_radians = np.radians(angles)

    mean_direction = circmean(angles_radians, low=0, high=2*np.pi)
    concentration = 1 - circstd(angles_radians, low=0, high=2*np.pi) / np.pi
    circular_variance = circvar(angles_radians, low=0, high=2*np.pi)

    direction_bins, bin_edges = np.histogram(angles, bins=36, range=(0, 360))
    primary_direction_index = np.argmax(direction_bins)
    primary_direction = (bin_edges[primary_direction_index] + bin_edges[primary_direction_index + 1]) / 2
    primary_direction_radians = np.radians(primary_direction)

    fig = Figure(figsize=(8, 6), dpi=100)
    ax = fig.add_subplot(111, projection='polar')
    
    if isinstance(chosen_color, LinearSegmentedColormap):
        bars = ax.hist(angles_radians, bins=36, color='skyblue', edgecolor='black', linewidth=1.2)
        for bar, angle in zip(bars[2], np.linspace(0, 2 * np.pi, len(bars[0]), endpoint=False)):
            bar.set_facecolor(chosen_color(angle / (2 * np.pi)))
    else:
        bars = ax.hist(angles_radians, bins=36, color=chosen_color, edgecolor='black', linewidth=1.2)

    ax.plot([mean_direction, mean_direction], [0, max(bars[0])], color='purple', linestyle='dotted', linewidth=2, label='Mean Direction')
    ax.plot([primary_direction_radians, primary_direction_radians], [0, max(bars[0])], color='purple', linestyle='dashed', linewidth=2, label='Primary Direction')

    ax.set_theta_direction(-1)
    ax.set_theta_zero_location('N')
    ax.set_title(title if title else 'Rose Diagram')

    ax.tick_params(axis='both', labelsize=12, colors='black')
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1))

    fig.tight_layout()

    label.config(text=f"Completed in {elapsed_time:.2f} seconds")
    progress['value'] = 100

    return fig, angles, bars

def show_plot(fig, angles, bars):
    for widget in root.winfo_children():
        if isinstance(widget, Frame):
            widget.destroy()

    plot_frame = Frame(root, bg='#f0f0f0')
    plot_frame.pack(expand=True, fill='both')

    canvas = FigureCanvasTkAgg(fig, master=plot_frame)
    canvas.draw()
    canvas.get_tk_widget().pack()

    results_frame = Frame(plot_frame, bg='#f0f0f0')
    results_frame.pack(pady=10)

    Button(results_frame, text="Mean Direction", command=lambda: calculate_mean_direction_display(angles), bg='#ff9800', fg='white').pack(side='left', padx=5)
    Button(results_frame, text="Concentration", command=lambda: calculate_concentration_display(angles), bg='#ff9800', fg='white').pack(side='left', padx=5)
    Button(results_frame, text="Circular Variance", command=lambda: calculate_circular_variance_display(angles), bg='#ff9800', fg='white').pack(side='left', padx=5)
    Button(results_frame, text="Export Plot", command=lambda: export_plot(fig), bg='#2196f3', fg='white').pack(side='left', padx=5)
    Button(results_frame, text="Export Statistics", command=lambda: export_statistics(angles, bars), bg='#4caf50', fg='white').pack(side='left', padx=5)

def create_rose():
    df = open_excel()
    if df is not None:
        title = simpledialog.askstring("Input", "Enter plot title:")
        chosen_color = choose_color()
        loading_screen(df, title, chosen_color, mode='manual')

def osm_mode():
    place_name = simpledialog.askstring("Input", "Enter place name for OSM data retrieval (e.g., 'Fatih, İstanbul, TR'):")
    if place_name:
        title = simpledialog.askstring("Input", "Enter plot title:")
        chosen_color = choose_color()
        loading_screen(place_name, title, chosen_color, mode='osm')

def loading_screen(data, title, chosen_color, mode='manual'):
    loading_window = Toplevel(root)
    loading_window.title("Processing")
    loading_window.geometry("300x100")

    progress = Progressbar(loading_window, orient='horizontal', length=200, mode='determinate')
    progress.pack(pady=20)

    label = Label(loading_window, text="Processing... 0%")
    label.pack()

    root.update()

    if mode == 'manual':
        fig, angles, bars = plot_rose(data, title, chosen_color, progress, label)
    else:
        fig, angles, bars = plot_osm_rose(data, title, chosen_color, progress, label)

    loading_window.destroy()
    show_plot(fig, angles, bars)

def show_main_menu():
    for widget in root.winfo_children():
        if isinstance(widget, Frame):
            widget.destroy()

    main_frame = Frame(root, bg='black')
    main_frame.pack(expand=True, fill='both')

    Button(main_frame, text="Manual Mode (Create Rose Diagram)", command=create_rose, bg='red', fg='white').pack(pady=20)
    Button(main_frame, text="OSM Mode", command=osm_mode, bg='blue', fg='white').pack(pady=20)

def start_gui():
    global root
    root = Tk()
    root.title("StreetRose 0.0.1")
    root.configure(bg='black')

    x = (root.winfo_screenwidth() // 2) - (WINDOW_WIDTH // 2)
    y = (root.winfo_screenheight() // 2) - (WINDOW_HEIGHT // 2)
    root.geometry(f"{WINDOW_WIDTH}x{WINDOW_HEIGHT}+{x}+{y}")

    splash_frame = Frame(root, bg='black')
    splash_frame.pack(expand=True)

    Label(splash_frame, text="Welcome to StreetRose 0.0.1", font=("Calibri", 25), bg='black', fg='red').pack(pady=20)
    Label(splash_frame, text=" Demo Version ", font=("Comic Sans MS", 15, "italic"), bg='black', fg='red').pack(pady=10)
    Label(splash_frame, text=ROSE_ASCII, font=("Courier", 10), bg='black', fg='red').pack(pady=20)

    root.after(3000, show_main_menu)

    root.mainloop()

if __name__ == "__main__":
    start_gui()
